# SQL Project
You were hired by Ironhack to perform an Analytics Consulting Project entitled: competitive landscape.

Your mission is to create and populate an appropriate database with many coding schools that are our competition, as well as design an suitable queries that answer business questions of interest (to be defined by you)


**Suggested Steps in the Project:**


*   Read this notebook and understand each function. Comment the code appropriately

*   Populate the list of schools with a wider variety of schools (how are you going to get the school ID?)

* Take a look at the obtained dataframes. What dimensions do you have? what keys do you have? how could the different dataframes be connected?

* Go back to the drawing board and try to create an entity relationship diagram for tables available

* Once you have the schemas you want, you will need to:
  - create the suitable SQL queries to create the tables and populate them
  - run these queries using the appropriate Python connectors
  
* Bonus: How will this datamodel be updated in the future? Please write auxiliary functions that test the database for data quality issues. For example: how could you make sure you only include the most recent comments when you re-run the script?


# Suggested Deliverables

* 7 minute presentation of data model created, decision process and business analysis proposed

* exported .sql file with the final schema

* Supporting python files used to generate all logic

* High level documentation explaining tables designed and focusing on update methods

Crucial hint: check out the following tutorial:
https://www.dataquest.io/blog/sql-insert-tutorial/


In [1]:
schools_main = {
#top 10 coding
'actualize': '10505',
'clarusway': '11539',   
'learningfuze': '10862',
'springboard': '11035',  
'codesmith': '10643',
'code-platoon': '10627',
'knowledgehut': '10846',   
'udacity': '11118', 
'altcademy': '10517',
'tech-elevator': '11056',
    
#top 10 Data
#2'springboard': '11035', 
'dataquest': '10683',
'syntax-technologies': '11797',
'ironhack': '10828',
'tripleten': '11225', 
'colaberry': '11718',
'maven-analytics': '11740',
#2'udacity': '11118', 
'brainstation': '10571',
'ccs-learning-academy': '11736',

#top 10 ux
'designlab': '10697',    
#3'springboard': '11035',   
#2'ironhack': '10828',
'ux-design-institute': '11150',
'mento-design-academy': '11584',
'beginex': '10546',
#3'udacity': '11118', 
#2'brainstation': '10571',
'design-sprint-school': '10698',
'general-assembly': '10761'}

import re
import pandas as pd
from pandas.io.json import json_normalize
import requests



def get_comments_school(school):
    TAG_RE = re.compile(r'<[^>]+>')
    # defines url to make api call to data -> dynamic with school if you want to scrape competition
    url = "https://www.switchup.org/chimera/v1/school-review-list?mainTemplate=school-review-list&path=%2Fbootcamps%2F" + school + "&isDataTarget=false&page=3&perPage=10000&simpleHtml=true&truncationLength=250"
    #makes get request and converts answer to json
    # url defines the page of all the information, request is made, and information is returned to data variable
    data = requests.get(url).json()
    #converts json to dataframe
    reviews =  pd.DataFrame(data['content']['reviews'])

    #aux function to apply regex and remove tags
    def remove_tags(x):
        return TAG_RE.sub('',x)
    reviews['review_body'] = reviews['body'].apply(remove_tags)
    reviews['school'] = school
    return reviews

In [2]:
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(x):
    return TAG_RE.sub('',x)

In [3]:
comments = [get_comments_school(i) for i in schools_main.keys()]

comments = pd.concat(comments)

In [4]:
to_drop = ["body", "name", "hostProgramName", "rawBody","queryDate", "user", "comments" ]
for i in to_drop:
    comments.drop(i, axis=1, inplace=True)
comments

,id,anonymous,graduatingYear,isAlumni,jobTitle,tagline,createdAt,program,overallScore,overall,curriculum,jobSupport,review_body,school
0,305645,True,2023.0,False,,Was great! They really teach you,10/13/2023,Actualize Coding Bootcamp: Online Live,5.0,5.0,5.0,5.0,Great program. They really teach how to code n...,actualize
1,305528,False,2023.0,False,,Full Stack Web Development Course,10/10/2023,Actualize Coding Bootcamp: Online Live,5.0,5.0,5.0,5.0,"Actualize's slogan is ""Educators first, develo...",actualize
2,304869,True,2023.0,True,,A great coding boot camp experience! Highly re...,9/14/2023,Actualize Coding Bootcamp: Online Live,5.0,5.0,5.0,5.0,There are so many great perks to this boot cam...,actualize
3,304868,False,2023.0,False,Senior IT Specialist,Exceptional Coding Boot Camp Experience,9/14/2023,Actualize Coding Bootcamp: Full-Time,5.0,5.0,5.0,5.0,I recently completed an inconceivable adventur...,actualize
4,304866,True,2023.0,True,,Phenomenal Bootcamp,9/14/2023,Actualize Coding Bootcamp: Online Live,5.0,5.0,5.0,5.0,Actualize is a well-thought-out curriculum wit...,actualize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,231691,False,2013.0,True,Software Developer at Cengage Learning,From 2nd grade teacher to full-stack web devel...,6/29/2014,Software Engineering Immersive,5.0,5.0,5.0,5.0,My experience at General Assembly's Web Develo...,general-assembly
995,231827,False,NaN,False,,What you get out of the program really depends...,6/15/2014,,3.0,3.0,NaN,NaN,What you get out of the program really depends...,general-assembly
996,231816,False,NaN,False,,The bitmaker program provides opportunities an...,6/15/2014,,5.0,5.0,NaN,NaN,Personally I had a great experience at Bitmake...,general-assembly
997,231836,False,NaN,False,,Bitmaker is the best way to motivate and accel...,6/15/2014,,5.0,5.0,NaN,NaN,I was in the 3rd cohort and have had a success...,general-assembly


In [5]:
from pandas.io.json import json_normalize

def get_school_info(school, school_id):
    url = 'https://www.switchup.org/chimera/v1/bootcamp-data?mainTemplate=bootcamp-data%2Fdescription&path=%2Fbootcamps%2F'+ str(school) + '&isDataTarget=false&bootcampId='+ str(school_id) + '&logoTag=logo&truncationLength=250&readMoreOmission=...&readMoreText=Read%20More&readLessText=Read%20Less'

    data = requests.get(url).json()

    data.keys()

    courses = data['content']['courses']
    courses_df = pd.DataFrame(courses, columns= ['courses'])

    locations = data['content']['locations']
    locations_df = json_normalize(locations)

    badges_df = pd.DataFrame(data['content']['meritBadges'])

    website = data['content']['webaddr']
    description = data['content']['description']
    logoUrl = data['content']['logoUrl']
    school_df = pd.DataFrame([website,description,logoUrl]).T
    school_df.columns =  ['website','description','LogoUrl']

    locations_df['school'] = school
    courses_df['school'] = school
    badges_df['school'] = school
    school_df['school'] = school


    locations_df['school_id'] = school_id
    courses_df['school_id'] = school_id
    badges_df['school_id'] = school_id
    school_df['school_id'] = school_id

    return locations_df, courses_df, badges_df, school_df

locations_list = []
courses_list = []
badges_list = []
schools_list = []

for school, id in schools_main.items():
    a,b,c,d = get_school_info(school,id)

    locations_list.append(a)
    courses_list.append(b)
    badges_list.append(c)
    schools_list.append(d)



C:\Users\Jacob\AppData\Local\Temp\ipykernel_8836\3406472284.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  locations_df = json_normalize(locations)
C:\Users\Jacob\AppData\Local\Temp\ipykernel_8836\3406472284.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  locations_df = json_normalize(locations)
C:\Users\Jacob\AppData\Local\Temp\ipykernel_8836\3406472284.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  locations_df = json_normalize(locations)
C:\Users\Jacob\AppData\Local\Temp\ipykernel_8836\3406472284.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  locations_df = json_normalize(locations)
C:\Users\Jacob\AppData\Local\Temp\ipykernel_8836\3406472284.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  locations_df = j

In [6]:
locations = pd.concat(locations_list)
locations

,id,description,country.id,country.name,country.abbrev,state.id,state.name,state.abbrev,state.keyword,city.id,city.name,city.keyword,school,school_id
0,15703,"Chicago, IL, United States",1.0,United States,US,15.0,Illinois,IL,illinois,39.0,Chicago,chicago,actualize,10505
1,16019,Online,NaN,NaN,NaN,1.0,Online,Online,online,NaN,NaN,NaN,actualize,10505
0,17789,Online,NaN,NaN,NaN,1.0,Online,Online,online,NaN,NaN,NaN,clarusway,11539
0,16712,"Irvine, CA, United States",1.0,United States,US,6.0,California,CA,california,24.0,Irvine,irvine,learningfuze,10862
1,16806,"San Diego, CA, United States",1.0,United States,US,6.0,California,CA,california,11.0,San Diego,san-diego,learningfuze,10862
2,16970,Online,NaN,NaN,NaN,1.0,Online,Online,online,NaN,NaN,NaN,learningfuze,10862
0,16013,Online,NaN,NaN,NaN,1.0,Online,Online,online,NaN,NaN,NaN,springboard,11035
0,16002,"Los Angeles, CA, United States",1.0,United States,US,6.0,California,CA,california,10.0,Los Angeles,los-angeles,codesmith,10643
1,16328,"New York City, NY, United States",1.0,United States,US,34.0,New York,NY,new-york,1507.0,NYC,new-york-city,codesmith,10643
2,16595,Online,NaN,NaN,NaN,1.0,Online,Online,online,NaN,NaN,NaN,codesmith,10643


In [7]:
courses = pd.concat(courses_list)
display(courses)

,courses,school,school_id
0,Actualize Coding Bootcamp: Full-Time,actualize,10505
1,Actualize Coding Bootcamp: Part-Time,actualize,10505
2,Actualize Coding Bootcamp: Online Live,actualize,10505
0,AWS Solutions Architect,clarusway,11539
1,AWS Solutions Architect - Premium Full Time/Pa...,clarusway,11539
...,...,...,...
9,React Development (Short Course),general-assembly,10761
10,Software Engineering Bootcamp (Full-Time),general-assembly,10761
11,User Experience Design (Short Course),general-assembly,10761
12,User Experience Design Bootcamp (Full-Time),general-assembly,10761


In [8]:
badges = pd.concat(badges_list)
badges["description"] = badges["description"].apply(remove_tags)
badges

,name,keyword,description,school,school_id
0,Available Online,available_online,School offers fully online courses,actualize,10505
1,Flexible Classes,flexible_classes,School offers part-time and evening classes,actualize,10505
0,Available Online,available_online,School offers fully online courses,clarusway,11539
1,Flexible Classes,flexible_classes,School offers part-time and evening classes,clarusway,11539
0,Available Online,available_online,School offers fully online courses,learningfuze,10862
...,...,...,...,...,...
1,Flexible Classes,flexible_classes,School offers part-time and evening classes,design-sprint-school,10698
0,Available Online,available_online,School offers fully online courses,general-assembly,10761
1,Verified Outcomes,verified_outcomes,School publishes a third-party verified outcom...,general-assembly,10761
2,Flexible Classes,flexible_classes,School offers part-time and evening classes,general-assembly,10761


In [9]:
schools = pd.concat(schools_list)

def schools_cleaning(row):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(row)
    text = [tag.get_text() for tag in soup.find_all(['p', 'span'])]
    return ' '.join(text)

schools["description"] = schools["description"].apply(schools_cleaning)

In [10]:
schools = schools.set_index("school").reset_index().set_index("school_id").reset_index()
schools

,school_id,school,website,description,LogoUrl
0,10505,actualize,anyonecanlearntocode.com/?utm_source=switchup&...,Actualize is a coding bootcamp that values qua...,https://d92mrp7hetgfk.cloudfront.net/images/si...
1,11539,clarusway,clarusway.com/,Clarusway is a highly respected IT training sc...,https://d92mrp7hetgfk.cloudfront.net/images/si...
2,10862,learningfuze,learningfuze.com,"LearningFuze is an immersive, 14-week web deve...",https://d92mrp7hetgfk.cloudfront.net/images/si...
3,11035,springboard,www.springboard.com/?utm_source=switchup&utm_m...,Springboard is an online learning platform tha...,https://d92mrp7hetgfk.cloudfront.net/images/si...
4,10643,codesmith,www.codesmith.io,Codesmith’s Software Engineering Immersive pro...,https://d92mrp7hetgfk.cloudfront.net/images/si...
5,10627,code-platoon,www.codeplatoon.org,Code Platoon is a 501(c)3 non-profit that help...,https://d92mrp7hetgfk.cloudfront.net/images/si...
6,10846,knowledgehut,www.knowledgehut.com,"KnowledgeHut is a leading training provider, h...",https://d92mrp7hetgfk.cloudfront.net/images/si...
7,11118,udacity,www.udacity.com/?utm_source=switchup&utm_mediu...,Udacity is the trusted market leader in talent...,https://d92mrp7hetgfk.cloudfront.net/images/si...
8,10517,altcademy,www.altcademy.com,Altcademy is a 100% online education institute...,https://d92mrp7hetgfk.cloudfront.net/images/si...
9,11056,tech-elevator,www.techelevator.com,Tech Elevator offers a full-stack coding bootc...,https://d92mrp7hetgfk.cloudfront.net/images/si...


In [ ]:
query=("""CREATE TABLE IF NOT EXISTS bootcamp.schools
            (school_id INT NOT NULL PRIMARY KEY,
            school_name VARCHAR(50) NOT NULL,
            website VARCHAR(250)
            )""")
cursor.execute(query)

query=("""CREATE TABLE IF NOT EXISTS bootcamp.programs
            (program_id INT NOT NULL PRIMARY KEY,
            program_name VARCHAR(50) NOT NULL,
            school_id INT NOT NULL,
            FOREIGN KEY (school_id) REFERENCES school(school_id)
            )""")
cursor.execute(query)

query=("""CREATE TABLE IF NOT EXISTS bootcamp.comments
            (comment_id INT NOT NULL PRIMARY KEY,
            anonymous boolean NOT NULL,
            year TINYINT,
            job_title VARCHAR(250),
            alumni NOT NULL boolean,
            comment_date date NOT NULL,
            program_id INT NOT NULL,
            overall_score float NOT NULL,
            curric_score float NOT NULL,
            job_score float NOT NULL,
            school_id INT NOT NULL,
            FOREIGN KEY (program_id) REFERENCES programs(program_id),
            FOREIGN KEY (school_id) REFERENCES schools(school_id)
            )""")
cursor.execute(query)

query = (""" CREATE TABLE IF NOT EXISTS bootcamp.location
            (location_id INT NOT NULL PRIMARY KEY, 
            country_id INT NOT NULL, 
            country_name VARCHAR(35) NOT NULL,
            city_id INT NOT NULL, 
            city_name VARCHAR(35) NOT NULL, 
            school_id INT NOT NULL,
            FOREIGN KEY (school_id) REFERENCES schools(school_id)
            )""")
cursor.execute(query)